In [ ]:
import pandas as pd
import glob
import datetime

In [ ]:
# CSV 파일 경로 리스트 생성
file_list = glob.glob("../../data/raw/KOBIS_박스오피스_영화정보/KC_KOBIS_BOX_OFFIC_MOVIE_INFO_*.csv")

# CSV 파일 데이터프레임으로 변환
df_list = [pd.read_csv(file) for file in file_list]

# CSV 파일 불러오기
m = pd.read_csv("../../data/raw/KOBIS_개봉일람.csv", encoding="cp949")

# 데이터프레임을 세로로 연결
merged_df = pd.concat(df_list, ignore_index=True)

merged_df = merged_df.rename(columns={i:j for i, j in zip(merged_df.columns, m.columns)})
m = m.rename(columns={i:j for i, j in zip(m.columns, merged_df.columns)})

m = pd.concat([m, merged_df], ignore_index=True, axis=0)

m

In [ ]:
# 전국 스크린수 컬럼에서 평균보다 큰 데이터만 선택
m = m[m["전국 스크린수"] > m["전국 스크린수"].mean()]

m

In [ ]:
# 순번, 수입사, 서울 매출액, 서울 관객수 컬럼 삭제
m = m.drop(["순번", "수입사", "서울 매출액", "서울 관객수"], axis=1)

In [ ]:
# 중복된 행 제거
m.drop_duplicates(subset=["영화명", "장르", "등급", "영화구분"], keep="first", inplace=True)
m

In [ ]:
# datetime 형식으로 변환하는 함수
def format_date(date):
    if isinstance(date, (int, float)):
        date = str(int(date))
    else:
        date = str(date)
    date = date.replace("-", "").replace(".", "")[:8]
    formatted_date = datetime.datetime.strptime(date[:6], "%Y%m")
    return formatted_date.strftime("%Y-%m")

# apply 함수를 사용하여 format_date 함수를 적용
m['개봉일'] = m['개봉일'].apply(format_date)

m

In [ ]:
m = m.drop(["영화유형"], axis=1)

In [ ]:
m = m.drop(["영화형태"], axis=1)

In [ ]:
# 국적 컬럼의 빈도수가 높은 상위 10개 값 선택
nation = "|".join(m["국적"].value_counts().head(10).index)

nation

In [ ]:
# 상위 10개 값이 아닌 값을 기타로 변경
m.loc[~m["국적"].str.contains(nation), "국적"] = "기타"

# 변경 후 기타의 수
etc_count = len(m[m["국적"] == "기타"])

# 변경후 상위 10개 값의 수
top10_count = len(m[m["국적"] != "기타"])

# 상위 10개 값의 수와 합쳐서 총 수 확인
print("전체 수:", top10_count + etc_count, len(m))

# 변경 후 고윳값 확인
print(m["국적"].unique())

In [ ]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "청소년관람불가15세이상관람가", "등급"] = "15세이상관람가"

m[m["등급"] == "청소년관람불가15세이상관람가"]

In [ ]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "12세관람가", "등급"] = "12세이상관람가"

m[m["등급"] == "12세관람가"]

In [ ]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "15세관람가", "등급"] = "15세이상관람가"

m[m["등급"] == "15세관람가"]

In [ ]:
# 검색 결과를 기반으로 값 변경
m.loc[m["등급"] == "12세이상관람가15세이상관람가", "등급"] = "12세이상관람가"

m[m["등급"] == "12세이상관람가15세이상관람가"]

In [ ]:
# 중복된 행 제거
m.drop_duplicates(subset=["영화명", "감독"], keep="first", inplace=True)
m

In [ ]:
m.to_csv("../../data/processed/processed_1.csv", index=False)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
m = pd.read_csv("../../data/processed/processed_1.csv")

m

In [ ]:
감독 = m[m["감독"].isna()]["영화명"].tolist()
제작사 = m[m["제작사"].isna()]["영화명"].tolist()
배급사 = m[m["배급사"].isna()]["영화명"].tolist()
관객수 = m[m["전국 매출액"].isna()]["영화명"].tolist()

In [ ]:
director_names = {}

for title in tqdm(감독):
    url = "https://search.naver.com/search.naver?ie=UTF-8&query=" + title + " 감독"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    try:
        director_name = soup.select_one("#mflick > div > div > ul > li > div > div.title_box > strong").text
        director_names[title] = director_name.strip()
    except:
        pass
    
director_names

In [ ]:
for k, v in director_names.items():
    m.loc[m["영화명"] == k, "감독"] = v
    print(m.loc[m["영화명"] == k][["영화명", "감독"]])

In [ ]:
production_company_list = {}

for title in tqdm(제작사):
    try:
        url = "https://ko.wikipedia.org/wiki/" + title
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        # 제작사 정보가 있는 Infobox 테이블의 label 태그를 선택
        infobox_label = soup.select(".infobox-label")

        # 제작사라는 텍스트를 가진 label 태그의 자손 th 태그 선택
        production_company_list[title] = "".join([label.find_next("a").text for label in infobox_label if label.text == "제작사"])
    except:
        pass

In [ ]:
for k, v in production_company_list.items():
    if v != "":
        m.loc[m["영화명"] == k, "제작사"] = v
        print(m.loc[m["영화명"] == k][["영화명", "제작사"]])

In [ ]:
배급사_리스트 = ["20세기 폭스 코리아", "UPI 코리아", "UPI 코리아", "UPI 코리아", "트리플픽쳐스", "인디플러그", "시네마리퍼블릭"]

for k, v in zip(배급사, 배급사_리스트):
    m.loc[m["영화명"] == k, "배급사"] = v
    print(m.loc[m["영화명"] == k][["영화명", "배급사"]])

In [ ]:
관객수_리스트 = [5274, 360873, 208739, 1966, 35210, 41049, 1100000]

for k, v in zip(관객수, 관객수_리스트):
    m.loc[m["영화명"] == k, "전국 관객수"] = v
    print(m.loc[m["영화명"] == k][["영화명", "전국 관객수"]])

In [ ]:
m.to_csv("../../data/processed/processed_2.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
m = pd.read_csv("../../data/processed/processed_2.csv")

m

In [ ]:
# 감독, 제작사 결측치 제거
m = m.dropna(subset=["감독", "제작사"]).reset_index(drop=True)

m

In [ ]:
# 감독별 전국 관객수 평균값 계산
director_groupby = m.groupby("감독")["전국 관객수"].apply(lambda x: x.sum().mean())
# 제작사별 전국 관객수 평균값 계산
producer_groupby = m.groupby("제작사")["전국 관객수"].apply(lambda x: x.sum().mean())
# 배급사별 전국 관객수 평균값 계산
distributor_groupby = m.groupby("배급사")["전국 관객수"].apply(lambda x: x.sum().mean())

# 각 값을 변환
m["감독"] = m["감독"].map(director_groupby)
m["제작사"] = m["제작사"].map(producer_groupby)
m["배급사"] = m["배급사"].map(distributor_groupby)

m

In [ ]:
# 연월 분리
m['개봉년'] = m['개봉일'].apply(lambda x: int(x.split('-')[0]))
m['개봉월'] = m['개봉일'].apply(lambda x: int(x.split('-')[1]))

# 영화명, 개봉일, 전국 매출액 컬럼 삭제
m = m.drop(["영화명", "개봉일", "전국 매출액"], axis=1)

m

In [ ]:
# 범주형 변수 추출
categorical_features = ["국적", "장르", "등급", "영화구분"]
m_categorical = m[categorical_features]

# 범주형 변수 OneHotEncoding
encoder = OneHotEncoder(handle_unknown='ignore')
m_categorical_encoded = encoder.fit_transform(m_categorical)

# OneHotEncoding 결과 데이터프레임으로 변환
m_categorical_encoded_df = pd.DataFrame.sparse.from_spmatrix(m_categorical_encoded, columns=encoder.get_feature_names_out(categorical_features))

# 열 이름 바꾸기
new_columns = [column.replace("x0_", "국적_").replace("x1_", "장르_").replace("x2_", "등급_").replace("x3_", "영화구분_") for column in m_categorical_encoded_df.columns]
m_categorical_encoded_df.columns = new_columns

# 인코딩 변수를 새로운 데이터프레임에 추가
m_new = pd.concat([m.drop(categorical_features, axis=1), m_categorical_encoded_df.reset_index(drop=True)], axis=1)

m_new

In [ ]:
# 개봉월 컬럼 삭제
m_new.drop(["개봉월"], axis=1, inplace=True)

m_new

In [ ]:
# 다중공선선 확인
X = m_new.iloc[:, :6]

# 변수(features)와 VIF 값을 담을 데이터프레임 생성
vif = pd.DataFrame()
vif["features"] = X.columns

# X 데이터프레임의 각 변수에 대한 VIF 값을 계산하여 vif_factor 컬럼에 저장
vif["vif_factor"] = [variance_inflation_factor(X.values.astype(float), i) for i in range(X.shape[1])]
vif

In [ ]:
# 상관관계 행렬
m_corr = X.corr()

# 상관관계 행렬로부터 공분산행렬 구하기
cov_matrix = np.linalg.inv(m_corr.values)

# 공분산행렬에서 대각성분 계산
eigenvalues = np.linalg.eigvals(cov_matrix)

# 다중공선성 진단을 위한 조건수(condition number) 계산
cond_num = np.max(eigenvalues) / np.min(eigenvalues)

cond_num

In [ ]:
m_new.to_csv("../../data/processed/processed_3.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve, validation_curve
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import multiprocessing
import optuna
import cma

plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
m = pd.read_csv("../../data/processed/processed_3.csv")

m

In [ ]:
def recursive_feature_elimination(X, y, model, min_features=1, verbose=True):
    # 초기 변수 개수
    n_features = X.shape[1]
    
    # 현재 변수들의 성능
    best_score = float("inf")
    best_features = X.columns.tolist()
    
    # 최소 변수 개수 이상인 경우
    while n_features > min_features:
        # 모든 변수에 대해 반복
        scores = []
        for feature in X.columns:
            # 선택한 변수 제외
            features = X.columns.drop(feature)
            X_new = X[features]
            
            # 모델 학습 및 평가
            model.fit(X_new, y)
            y_pred = model.predict(X_new)
            score = mean_squared_error(y, y_pred)
            scores.append(score)
            
        # 가장 성능이 좋은 변수 선택
        idx = pd.Index(scores).argmin()
        worst_feature = X.columns[idx]
        
        # 변수 제거
        X.drop(worst_feature, axis=1, inplace=True)
        n_features = X.shape[1]
        if verbose:
            print(f"Removing {worst_feature}: {n_features} features left")
        
        # 모든 변수를 제거한 경우
        if n_features == 0:
            break
        
        # 현재 변수 개수에서의 성능이 최선인 경우
        if min(scores) < best_score:
            best_score = min(scores)
            best_features = X.columns.tolist()
        else:
            break
    
    # 최종 선택된 변수들
    return best_features

In [ ]:
# ElasticNetCV 모델 객체 생성
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
enet = ElasticNetCV(alphas=alphas, l1_ratio=0.5, cv=5, max_iter=10000)

# 최소 변수 개수 지정
min_features = 5

# 재귀적 변수 제거 수행
X = m.drop("전국 관객수", axis=1)
y = m["전국 관객수"]
selected_features = recursive_feature_elimination(X, y, enet, min_features=min_features)

# 선택된 변수 출력
print(f"{len(selected_features)} features selected:")
print(selected_features)

In [ ]:
# 데이터 분할
X = m[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# RandomForestRegressor 모델 학습
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# RandomForestRegressor 모델 예측
rf_y_pred = rf.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
mae = mean_absolute_error(y_test, rf_y_pred)
r2 = r2_score(y_test, rf_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
def plot_learning_curve(model, X, y, cv=5, scoring="r2"):
    # 학습곡선 계산
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=cv, scoring=scoring,
                                                            train_sizes=np.linspace(0.1, 1.0, 10))

    # 트레이닝, 크로스벨리데이션 평균 계산
    train_mean = np.mean(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)

    # 학습곡선 시각화
    plt.plot(train_sizes, train_mean, label="트레이닝 점수")
    plt.plot(train_sizes, test_mean, label="검증 점수")
    plt.title(f"{type(model).__name__} 학습곡선")
    plt.xlabel("학습 데이터 수")
    plt.ylabel("성능 점수")
    plt.legend()
    plt.show()

In [ ]:
# 학습곡선 그리기
plot_learning_curve(rf, X_train, y_train)

In [ ]:
# GradientBoostingRegressor 모델 학습
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)

# GradientBoostingRegressor 모델 예측
gb_y_pred = gb.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, gb_y_pred, squared=False)
mae = mean_absolute_error(y_test, gb_y_pred)
r2 = r2_score(y_test, gb_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(gb, X_train, y_train)

In [ ]:
# XGBRegressor 모델 학습
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

# XGBRegressor 모델 예측
xgb_y_pred = xgb.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, xgb_y_pred, squared=False)
mae = mean_absolute_error(y_test, xgb_y_pred)
r2 = r2_score(y_test, xgb_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(xgb, X_train, y_train)

In [ ]:
# LGBMRegressor 모델 학습
lgbm = LGBMRegressor(random_state=42)
lgbm.fit(X_train, y_train)

# LGBMRegressor 모델 예측
lgbm_y_pred = lgbm.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, lgbm_y_pred, squared=False)
mae = mean_absolute_error(y_test, lgbm_y_pred)
r2 = r2_score(y_test, lgbm_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(lgbm, X_train, y_train)

In [ ]:
# ExtraTreesRegressor 모델 학습
et = ExtraTreesRegressor(random_state=42)
et.fit(X_train, y_train)

# ExtraTreesRegressor 모델 예측
et_y_pred = et.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, et_y_pred, squared=False)
mae = mean_absolute_error(y_test, et_y_pred)
r2 = r2_score(y_test, et_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(et, X_train, y_train)

In [ ]:
# 각 모델에서 예측한 결과값을 변수로 갖는 데이터프레임 생성
pred_df = pd.DataFrame({
        "rf": rf_y_pred,
        "gb": gb_y_pred,
        "sgb": xgb_y_pred,
        "lgbm": lgbm_y_pred,
        "et" : et_y_pred
    })

# 상관 행렬 계산
corr_matrix = pred_df.corr()

# 시각화
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.show()

In [ ]:
# 메타 모델 선정
models = [rf, gb, xgb, lgbm, et]
best_score = float("-inf")
best_model = None

for model in models:
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    if rmse > best_score:
        best_score = rmse
        best_model = model

print("best_model :", best_model)

# 앙상블 모델 정의
stack = StackingCVRegressor(regressors=(rf, gb, xgb, lgbm, et),
                            meta_regressor=best_model,
                            cv=KFold(n_splits=5, shuffle=True, random_state=42),
                            use_features_in_secondary=True)

# 앙상블 모델 학습
stack.fit(X_train, y_train)

# 앙상블 모델 예측
y_pred = stack.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(stack, X_train, y_train)

In [ ]:
def optimize_model(model_objective, n_trials=100):
    # Bayesian optimization 방식의 하이퍼파라미터 튜닝을 위해 Optuna 라이브러리 사용
    # 최소화를 목적으로 하기 때문에 'minimize'로 설정
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.CmaEsSampler())
    
    # 사용 가능한 모든 CPU 코어 수를 사용하여 병렬 처리
    n_jobs = multiprocessing.cpu_count()  
    
    # 모델 하이퍼파라미터 최적화 실행
    study.optimize(model_objective, n_trials=n_trials, n_jobs=n_jobs)
    
    # 최적 하이퍼파라미터와 그 때의 평가지표 출력
    print(f"Best RMSE: {study.best_value:.4f}")
    print(f"Best Parameters: {study.best_params}")
    
    # 최적 하이퍼파라미터 반환
    return study.best_params

In [ ]:
# Random Forest 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def rf_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    score = cross_val_score(rf, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# Random Forest 모델의 하이퍼파라미터 최적화
rf_params = optimize_model(rf_objective)

In [ ]:
# 최적의 하이퍼 파라미터 적용 후 학습 및 예측
rf.set_params(**rf_params)
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
mae = mean_absolute_error(y_test, rf_y_pred)
r2 = r2_score(y_test, rf_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(rf, X_train, y_train)

In [ ]:
# Gradient Boosting 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def gb_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    gb = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
                                   max_depth=max_depth, random_state=42)
    score = cross_val_score(gb, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# Gradient Boosting 모델의 하이퍼파라미터 최적화
gb_params = optimize_model(gb_objective)

In [ ]:
# 최적의 하이퍼 파라미터 적용 후 학습 및 예측
gb.set_params(**gb_params)
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, gb_y_pred, squared=False)
mae = mean_absolute_error(y_test, gb_y_pred)
r2 = r2_score(y_test, gb_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(gb, X_train, y_train)

In [ ]:
# XGBoost 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def xgb_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    xgb = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
                       max_depth=max_depth, subsample=subsample, random_state=42)
    score = cross_val_score(xgb, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse
    
# XGBoost 모델의 하이퍼파라미터 최적화
xgb_params = optimize_model(xgb_objective)

In [ ]:
# 최적의 하이퍼 파라미터 적용 후 학습 및 예측
xgb.set_params(**xgb_params)
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, xgb_y_pred, squared=False)
mae = mean_absolute_error(y_test, xgb_y_pred)
r2 = r2_score(y_test, xgb_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(xgb, X_train, y_train)

In [ ]:
# LightGBM 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def lgbm_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    lgbm = LGBMRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
    max_depth=max_depth, num_leaves=num_leaves, random_state=42)
    score = cross_val_score(lgbm, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# LightGBM 모델의 하이퍼파라미터 최적화
lgbm_params = optimize_model(lgbm_objective)

In [ ]:
# 최적의 하이퍼 파라미터 적용 후 학습 및 예측
lgbm.set_params(**lgbm_params)
lgbm.fit(X_train, y_train)
lgbm_y_pred = lgbm.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, lgbm_y_pred, squared=False)
mae = mean_absolute_error(y_test, lgbm_y_pred)
r2 = r2_score(y_test, lgbm_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(lgbm, X_train, y_train)

In [ ]:
# Extra Trees Regressor 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def et_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    et = ExtraTreesRegressor(n_estimators=n_estimators, max_depth=max_depth, 
                             min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, 
                             random_state=42)
    score = cross_val_score(et, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# Extra Trees Regressor 모델의 하이퍼파라미터 최적화
et_params = optimize_model(et_objective)

In [ ]:
# 최적의 하이퍼 파라미터 적용 후 학습 및 예측
et.set_params(**et_params)
et.fit(X_train, y_train)
et_y_pred = et.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, et_y_pred, squared=False)
mae = mean_absolute_error(y_test, et_y_pred)
r2 = r2_score(y_test, et_y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(et, X_train, y_train)

In [ ]:
# 메타 모델 선정
models = [rf, gb, xgb, lgbm, et]
best_score = float("-inf")
best_model = None

for model in models:
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    if rmse > best_score:
        best_score = rmse
        best_model = model

print("best_model :", best_model)

# 앙상블 모델 정의
stack = StackingCVRegressor(regressors=(rf, gb, xgb, lgbm, et),
                            meta_regressor=best_model,
                            cv=KFold(n_splits=5, shuffle=True, random_state=42),
                            use_features_in_secondary=True)

# 앙상블 모델 학습
stack.fit(X_train, y_train)

# 앙상블 모델 예측
y_pred = stack.predict(X_test)

# 평가지표 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

In [ ]:
# 학습곡선 그리기
plot_learning_curve(stack, X_train, y_train)